# TEXT CLASSIFICATION WITH TORCHTEXT

In [8]:

# import urllib.request as urlreq
# # 设置https代理
# ph = urlreq.ProxyHandler({'https': 'https://127.0.0.1:10809'})
# oper = urlreq.build_opener(ph)
# # 将代理安装到全局环境，这样所有请求都会自动使用代理
# urlreq.install_opener(oper)

import os
os.environ['http_proxy'] = 'http://127.0.0.1:10809'
os.environ['https_proxy'] = 'https://127.0.0.1:10809'

In [9]:
import torch
import torchtext
from torchtext.datasets import text_classification

NGRAMS = 2

import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
    
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data',ngrams=NGRAMS, vocab=None,
)

BATCH_SIZE = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

120000lines [00:19, 6190.14lines/s]
120000lines [00:38, 3121.23lines/s]
7600lines [00:02, 2798.48lines/s]


In [15]:
import torch.nn as nn
import torch.functional as F

class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)
    

AG_NEWS 一共有四个labels，因此可以分为四类
```
: World
: Sports
: Business
: Sci/Tec
```

In [16]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUM_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)


生成batch的函数

因为文本条目的长度不一样，因此需要一个`generate_batch()`来生成数据和偏移量

In [17]:
def generate_batch(batch):
    label = torch.tensor([entry[0]  for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text] # 求出每个长度
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) # 累积后变成 偏移量
    text = torch.cat(text) # 将文本连接起来
    return text, offsets, label

定义 训练函数 和 评估函数

In [18]:
from torch.utils.data import DataLoader

def train_func(sub_train_):
    
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [19]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 1 minutes, 4 seconds
	Loss: 0.0261(train)	|	Acc: 84.8%(train)
	Loss: 0.0000(valid)	|	Acc: 90.1%(valid)
Epoch: 2  | time in 1 minutes, 7 seconds
	Loss: 0.0120(train)	|	Acc: 93.6%(train)
	Loss: 0.0000(valid)	|	Acc: 90.8%(valid)
Epoch: 3  | time in 0 minutes, 56 seconds
	Loss: 0.0069(train)	|	Acc: 96.4%(train)
	Loss: 0.0000(valid)	|	Acc: 91.5%(valid)
Epoch: 4  | time in 0 minutes, 57 seconds
	Loss: 0.0039(train)	|	Acc: 98.1%(train)
	Loss: 0.0000(valid)	|	Acc: 91.0%(valid)
Epoch: 5  | time in 0 minutes, 51 seconds
	Loss: 0.0023(train)	|	Acc: 99.0%(train)
	Loss: 0.0000(valid)	|	Acc: 91.3%(valid)


In [20]:
torch.save(model,'201model.pkl')

/home/dongxian/miniconda3/envs/py37/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type TextSentiment. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
